In [1]:
import json
import lightgbm as lgb
import numpy as np
import optuna
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as pyo
import tensorflow as tf
import warnings
import xgboost as xgb
from catboost import CatBoost
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from keras.backend import clear_session
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, LSTM, Dropout, Flatten, Conv1D, TimeDistributed, RepeatVector
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.models import Sequential, load_model
from keras.optimizers import Adam
warnings.simplefilter(action='ignore', category=(FutureWarning, UserWarning, RuntimeWarning))

In [2]:
# Configuração do notebook para rodar algoritmos de tunagem e/ou treinamento
TUNE = True
n_trials = 50
TRAIN = True

# Funções auxiliares

In [3]:
# Função para criar dados multivariados
def multivariate_data(
    dataset: np.ndarray,
    target: np.ndarray,
    start_index: int,
    end_index: int,
    history_size: int,
    target_size: int,
    step: int,
    single_step: bool = False
) -> tuple[np.ndarray, np.ndarray]:
    """Create multivariate data for time series forecasting.

    This function generates input data and corresponding target data for time series forecasting tasks.
    It can be used for both single-step and multi-step forecasting.
    The function is based on the TensorFlow tutorial: https://www.tensorflow.org/tutorials/structured_data/time_series#part_2_forecast_a_multivariate_time_series 

    Args:
        dataset (np.ndarray): The input dataset, typically a 2D array with shape (num_samples, num_features).
        target (np.ndarray): The target variable or values to predict, typically a 1D array with shape (num_samples,).
        start_index (int): The starting index of the data in the dataset array.
        end_index (int): The ending index of the data in the dataset array.
        history_size (int): The number of time steps to consider as history for each input data point.
        target_size (int): The number of future time steps to predict.
        step (int): The number of time steps to skip between data points in the history.
        single_step (bool, optional): If True, create data for single-step forecasting; if False, create data for multi-step forecasting. Defaults to False.

    Returns:
        tuple[np.ndarray, np.ndarray]: A tuple containing:
            - A 3D array of shape (num_samples, history_size, num_features) containing input data.
            - A 1D or 2D array (depending on single_step) containing target data.
              If single_step is True, it's a 1D array of shape (num_samples,) with the next time step's target.
              If single_step is False, it's a 2D array of shape (num_samples, target_size) with multi-step targets.

    Example:
        >>> data, labels = multivariate_data(
        ...     dataset=data_array,
        ...     target=target_array,
        ...     start_index=0,
        ...     end_index=100,
        ...     history_size=10,
        ...     target_size=1,
        ...     step=1,
        ...     single_step=True
        ... )
    """
    data = []
    labels = []
    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size
    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])
        if single_step:
            labels.append(target[i + target_size])
        else:
            labels.append(target[i : i + target_size])

    return np.array(data), np.array(labels)

In [4]:
def metric_display(y_test, y_pred, idx_start = None, idx_end = None) -> list[float] :
    """Exibe métricas de avaliação de um modelo de regressão e gera um gráfico de comparação.

    Esta função calcula várias métricas de avaliação de um modelo de regressão, incluindo RMSE, MAE, MAPE, SMAPE, R² e EVS.
    Além disso, ela cria um gráfico de linha para visualizar a comparação entre os valores reais (y_test) e os valores previstos (y_pred).

    Args:
        y_test (array-like): Os valores reais da variável de destino.
        y_pred (array-like): Os valores previstos pelo modelo.
        idx_start (str): A data e hora de início do período de teste.
        idx_end (str): A data e hora de término do período de teste.

    Returns:
        list[float]

    Examples:
        >>> y_test = [3.0, 4.0, 5.0, 6.0, 7.0]
        >>> y_pred = [2.8, 4.2, 4.9, 6.3, 7.2]
        >>> metric_display(y_test, y_pred, idx_start='2021-01-01 00:00', idx_end='2021-01-01 04:00')
        
    """
    # Calcula o erro médio quadrático (RMSE)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    # Calcula o erro absoluto médio (MAE)
    mae = mean_absolute_error(y_test, y_pred)
    
    # Calcula o erro percentual absoluto médio (MAPE)
    mape = np.mean(np.abs((y_pred - y_test) / y_test)) * 100
    
    # Calcula o erro percentual absoluto médio simétrico (SMAPE)
    smape = 2 * np.mean(np.abs(y_pred - y_test) / (np.abs(y_pred) + np.abs(y_test))) * 100
    
    # Calcula o coeficiente de determinação (R²)
    r_squared = r2_score(y_test, y_pred)
    
    # Calcula a pontuação de variância explicada (EVS)
    evs = explained_variance_score(y_test, y_pred)

    print(f'[Menor é melhor] RMSE: {rmse:.4f}')
    print(f'[Menor é melhor] MAE: {mae:.4f}')
    print(f'[Menor é melhor] MAPE: {mape:.4f}%')
    print(f'[Menor é melhor] SMAPE: {smape:.4f}% ')
    print(f'[Maior é melhor] R²: {r_squared:.4f} ')
    print(f'[Maior é melhor] EVS: {evs:.4f} ')

    # Cria um índice para o período de teste com base nos parâmetros fornecidos
    if idx_start is not None and idx_end is not None:
        # Se as datas forem fornecidas, use-as para criar o índice de hora em hora
        idx = pd.date_range(start=idx_start, end=idx_end, freq='H')
    else:
        # Caso contrário, use índices numéricos padrão com base no comprimento dos dados
        idx = range(len(y_test))

    # Cria um DataFrame com o índice calculado
    df = pd.DataFrame({'Real (y_test)': y_test.ravel(), 'Previsto (y_pred)': y_pred.ravel()}, index=idx)

    # Gera um gráfico de comparação
    fig = px.line(df, title='Comparação entre y_test e y_pred',
                   labels={'value': 'Valor', 'variable': 'Valor'},
                   line_shape='linear')
    if idx_start is not None and idx_end is not None:
        fig.update_xaxes(title_text='Data e Hora', tickformat="%Y-%m-%d %H:%M")

    # Exibe o gráfico
    fig.show()

    # retorna as métricas calculadas em uma lista
    return [rmse, mae, mape, smape, r_squared, evs]

In [5]:
def plot_training_history(history):
    """
    Plota a história de treinamento de um modelo Keras usando Plotly. 

    Args:
        history (keras.callbacks.History): O objeto de história de treinamento do modelo Keras.

    Returns:
        None
    """
    # Extrair informações da história de treinamento
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Converter o objeto de intervalo em lista
    epochs = list(range(1, len(train_loss) + 1))

    # Criar um gráfico interativo com proporção quadrada
    fig = go.Figure()

    # Adicionar curva de perda de treinamento
    fig.add_trace(go.Scatter(x=epochs, y=train_loss, mode='lines', name='Train Loss'))

    # Adicionar curva de perda de validação
    fig.add_trace(go.Scatter(x=epochs, y=val_loss, mode='lines', name='Validation Loss'))

    # Configurar o layout do gráfico
    fig.update_layout(
        title='Loss Over Epochs',
        xaxis_title='Epochs',
        yaxis_title='Loss',
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True),
        width=1000,
        height=600
    )

    # Exibir o gráfico
    fig.show()

# Carregar dados

In [6]:
# Carregar os dados
data = pd.read_csv('../TCC/datasets/forecast_dap.csv')
data['Timestamp'] = pd.to_datetime(data['Timestamp']).dt.strftime('%Y-%m-%d %H:%M')
data.set_index('Timestamp', inplace=True)

# Remover os últimos 24 dados para futura validação da previsão
future_data = data.tail(24)
data = data.iloc[:-24]

In [7]:
# Apresentar as colunas do dataset
data.columns

Index(['Day Ahead Price', 'Biomass', 'Fossil Gas', 'Fossil Hard coal',
       'Fossil Oil', 'Hydro Pumped Storage', 'Hydro Run-of-river and poundage',
       'Hydro Water Reservoir', 'Nuclear', 'Solar', 'Waste', 'Wind Onshore',
       'Total Generation', 'Actual Load', 'BE', 'CH', 'DE_AT_LU', 'ES', 'GB',
       'IT_NORD', 'IT_NORD_FR', 'Total Flux', 'DE_LU', 'Weekday', 'Holiday'],
      dtype='object')

In [8]:
# Definindo a coluna alvo
target = 'Day Ahead Price'

# Seleção de features e Preparação para a modelagem

In [9]:
# Copiando o dataframe original para preservar os dados originais
df_final = data.copy()

# Definir o caminho para salvar os modelos
model_path = f'../TCC/modelos/{target.replace(" ", "_")}_'

# Criação dos conjuntos de treinamento, validação e teste
train_end_idx = df_final.index.get_loc('2022-12-01 00:00')  # Índice de término para treinamento
valid_end_idx = df_final.index.get_loc('2023-02-01 00:00')  # Índice de término para validação
test_end_idx = df_final.index.get_loc(data.index[-1])   # Índice de término para teste

# Separação dos dados de entrada e saída
X = df_final[df_final.columns.drop(target)].values # Dados de entrada
y = df_final[target].values.reshape(-1, 1)  # Dados de saída

# Inicialização e ajuste dos scalers para normalização dos dados
scaler_X = MinMaxScaler(feature_range=(0, 1))   # Inicialização do scaler para normalização dos dados de entrada
scaler_y = MinMaxScaler(feature_range=(0, 1))   # Inicialização do scaler para normalização dos dados de saída
scaler_X.fit(X[:train_end_idx]) # Ajuste ao conjunto de treinamento
scaler_y.fit(y[:train_end_idx]) # Ajuste ao conjunto de treinamento
X_norm = scaler_X.transform(X)  # Normalização dos dados de entrada
y_norm = scaler_y.transform(y)  # Normalização dos dados de saída

# Redução de dimensionalidade usando PCA
pca = PCA().fit(X_norm[:train_end_idx]) # Ajuste aos dados normalizados de entrada
num_components = len(pca.explained_variance_ratio_) # Número de componentes principais

# Plotagem da variância explicada pelo número de componentes principais
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(1, num_components+1), y=np.cumsum(pca.explained_variance_ratio_), mode='lines', name='Cumulativa', marker_color='gray'))
fig.add_trace(go.Bar(x=np.arange(1, num_components+1), y=pca.explained_variance_ratio_, name='Individual', marker_color='blue'))
fig.update_layout(title='Variância explicada pelo número de componentes principais',
                  xaxis_title='Número de componentes principais',
                  yaxis_title='Variância explicada',
                  autosize=False, width=1000, height=600)
fig.show()

# Redução de dimensionalidade usando PCA com 80% de variância explicada mantida
pca = PCA(n_components=0.80).fit(X_norm[:train_end_idx]) # Ajuste aos dados normalizados de entrada
X_pca = pca.transform(X_norm) # Transformação dos dados normalizados de entrada

# Concatenação das características reduzidas pelo PCA e dos alvos normalizados
dataset_norm = np.concatenate((X_pca, y_norm), axis=1) # Concatenação das características reduzidas pelo PCA e dos alvos normalizados

# Definição do tamanho da janela de histórico e do tamanho do alvo futuro
past_history = 24 # Número de horas anteriores utilizado para prever o dado futuro
future_target = 0 # Se for 0, o modelo será de regressão, caso contrário, será de classificação

# Criação de conjuntos de treinamento, validação e teste usando a função 'multivariate_data'
X_train, y_train = multivariate_data(dataset_norm, dataset_norm[:, -1], 0, train_end_idx, past_history, future_target, step=1, single_step=True)
X_val, y_val = multivariate_data(dataset_norm, dataset_norm[:, -1], train_end_idx, valid_end_idx, past_history, future_target, step=1, single_step=True)
X_test, y_test = multivariate_data(dataset_norm, dataset_norm[:, -1], valid_end_idx, test_end_idx, past_history, future_target, step=1, single_step=True)

# Definição do tamanho do lote e do tamanho do buffer
batch_size = 32 # Número de amostras por atualização de gradiente (batch), menor mais lento, mas mais preciso (evita overfitting) 
buffer_size = 1000 # Tamanho do buffer para embaralhar os dados de treinamento, utilizado para datasets muito grandes

# Criação de conjunto de dados TensorFlow para treinamento e validação
# tf.data.Dataset.from_tensor_slices() cria um conjunto de dados a partir de um tensor
# cache() mantém os dados na memória após serem carregados do disco durante a primeira época
# shuffle() embaralha os dados para garantir que a ordem dos dados não afete o treinamento
# batch() agrupa os dados em lotes de tamanho igual
# prefetch() prepara os dados para a próxima iteração
train = tf.data.Dataset.from_tensor_slices((X_train, y_train)).cache().shuffle(buffer_size).batch(batch_size).prefetch(1) # Conjunto de treinamento
validation = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size).prefetch(1) # Conjunto de validação

# Definição de parâmetros comuns aos modelos
input_shape = X_train.shape[-2:] # Formato dos dados de entrada (número de amostras, número de características)
loss = MeanSquaredError() # Função de perda (erro quadrático médio)
metric = [RootMeanSquaredError()] # Métrica (raiz quadrada do erro quadrático médio)
early_stopping = EarlyStopping(patience=10) # Parada antecipada (sem melhoria no erro quadrático médio por 10 épocas)

# Reformulação de 'y_test' para sua forma original e inversão da normalização
y_test = y_test.reshape(-1, 1) # Reformulação de 'y_test' para sua forma original
y_test_inv = scaler_y.inverse_transform(y_test) # Inversão da normalização de 'y_test'

# Criar um DataFrame que irá armazenar as métricas de avaliação de cada modelo
model_metrics = pd.DataFrame(columns=['MODEL','RMSE', 'MAE', 'MAPE', 'SMAPE', 'R²', 'EVS'])

# Catboost

In [10]:
# Reformular os dados de treinamento, validação e teste para o formato do CatBoost
X_train_catboost = X_train.reshape(-1, X_train.shape[1] * X_train.shape[2])
X_val_catboost = X_val.reshape(-1, X_val.shape[1] * X_val.shape[2])
X_test_catboost = X_test.reshape(-1, X_test.shape[1] * X_test.shape[2])

In [11]:
if TUNE == True:
    # Defina a função de objetivo para otimização do Optuna
    def objective(trial):
        # Defina os parâmetros a serem otimizados
        params = {
            "loss_function": "RMSE",
            "random_state": 42,
            "iterations": trial.suggest_int("iterations", 100, 1000),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
            "depth": trial.suggest_int("depth", 1, 10),
            "subsample": trial.suggest_float("subsample", 0.05, 1.0),
            "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        }

        # Crie um modelo CatBoostRegressor com os parâmetros definidos
        model = CatBoostRegressor(**params, verbose=False)

        # Treine o modelo
        model.fit(X_train_catboost, y_train.ravel())

        # Faça previsões para o conjunto de validação
        forecast = model.predict(X_val_catboost)

        # Calcule o erro quadrático médio
        rmse = np.sqrt(mean_squared_error(y_val, forecast))
    
        return rmse

    # Crie um estudo Optuna com o pruner
    study = optuna.create_study(direction="minimize") # Minimize o RMSE
    study.optimize(objective, n_trials=n_trials, n_jobs=-1)  # Execute n_trials tentativas de otimização

    # Obtenha os melhores hiperparâmetros encontrados
    best_params = study.best_params

    # Adicionando os parâmetros fixos, que não foram otimizados pelo Optuna
    best_params.update({
        "loss_function": "RMSE",
        "random_state": 42,
    })

    # Salvando os hiperparâmetros do modelo em um arquivo JSON
    with open(f"{model_path}catboost_hyperparameters.json", "w") as f:
        json.dump(best_params, f)

    # Imprimir os melhores hiperparâmetros
    print("Melhores Hiperparâmetros:", best_params)

    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

    fig = optuna.visualization.plot_timeline(study)
    fig.show()

In [12]:
if TRAIN == True:
    # Carrega os melhores hiperparâmetros do modelo a partir do arquivo JSON
    with open(f"{model_path}catboost_hyperparameters.json", "r") as f:
        best_params = json.load(f)

    # Imprimir os melhores hiperparâmetros
    print("Hiperparâmetros Carregados:")
    for key, value in best_params.items(): print(f"{key}: {value}")
    
    # Use os melhores hiperparâmetros para treinar o modelo final
    best_catboost_model = CatBoostRegressor(**best_params)
    best_catboost_model.fit(X_train_catboost, y_train.ravel())

    # Salvar o modelo treinado
    best_catboost_model.save_model(f"{model_path}catboost_model.cbm")

In [13]:
# Carregar o modelo CatBoost
catboost_model = CatBoost()
catboost_model.load_model(f"{model_path}catboost_model.cbm")

# Geração de previsões usando o modelo carregado
forecast = catboost_model.predict(X_test_catboost)

# Reformulação das previsões para o formato adequado e inversão da normalização
forecast = forecast.reshape(-1, 1)
cat_pred = scaler_y.inverse_transform(forecast)

In [14]:
# Exibição das métricas de avaliação e do gráfico de comparação
resultado = metric_display(y_test_inv, cat_pred, df_final.index[valid_end_idx+past_history], df_final.index[test_end_idx-1])

# Adicionando as métricas de avaliação ao DataFrame, se o modelo já estiver no DataFrame, atualize as métricas, caso contrário, adicione uma nova linha
model_name = 'CatBoost'
if model_name in model_metrics['MODEL'].values:
    model_metrics.loc[model_metrics['MODEL'] == model_name] = [model_name] + resultado
else:
    model_metrics.loc[len(model_metrics)] = [model_name] + resultado

[Menor é melhor] RMSE: 248.1146
[Menor é melhor] MAE: 168.7578
[Menor é melhor] MAPE: inf%
[Menor é melhor] SMAPE: 13.2837% 
[Maior é melhor] R²: 0.7508 
[Maior é melhor] EVS: 0.7548 


# LGBM

In [10]:
# Reformular os dados de treinamento, validação e teste
X_train_lgb = X_train.reshape(-1, X_train.shape[1] * X_train.shape[2]) # Reformulação dos dados de treinamento para terem apenas 2 dimensões
X_val_lgb = X_val.reshape(-1, X_val.shape[1] * X_val.shape[2]) # Reformulação dos dados de validação para terem apenas 2 dimensões
X_test_lgb = X_test.reshape(-1, X_test.shape[1] * X_test.shape[2]) # Reformulação dos dados de teste para terem apenas 2 dimensões

In [11]:
if TUNE == True:
    # Defina a função de objetivo para otimização do Optuna
    def objective(trial):
        
        # Defina os parâmetros a serem otimizados
        params = {
            "objective": "regression",
            "metric": "rmse",
            "random_state": 42,
            "force_col_wise": True,
            "device": "gpu",
            "gpu_platform_id": 1,
            "gpu_device_id": 0,
            "num_leaves": trial.suggest_int("num_leaves", 2, 64),
            "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
            "n_estimators": trial.suggest_int("n_estimators", 100, 1000)
        }
        # Crie um modelo LGBMRegressor com os parâmetros definidos
        model = LGBMRegressor(**params, verbosity=-1)

        # Treine o modelo
        model.fit(X_train_lgb, y_train.ravel())

        # Faça previsões para o conjunto de validação
        forecast = model.predict(X_val_lgb)

        # Calcule o erro quadrático médio
        rmse = np.sqrt(mean_squared_error(y_val, forecast))

        return rmse

    # Crie um estudo Optuna e otimize a função objetivo
    study = optuna.create_study(direction="minimize")  # Minimize o RMSE
    study.optimize(objective, n_trials=n_trials, n_jobs=-1)  # Execute n_trials tentativas de otimização

    # Obtenha os melhores hiperparâmetros encontrados
    best_params = study.best_params

    # Adicionando os parâmetros fixos, que não foram otimizados pelo Optuna
    best_params.update({
        "objective": "regression",
        "metric": "rmse",
        "random_state": 42,
        "force_col_wise": True,
        "device": "gpu",
        "gpu_platform_id": 1,
        "gpu_device_id": 0,
    })

    # Salvando os hiperparâmetros do modelo em um arquivo JSON
    with open(f"{model_path}lgbm_hyperparameters.json", "w") as f:
        json.dump(best_params, f)

    # Imprimir os melhores hiperparâmetros
    print("Melhores Hiperparâmetros:", best_params)

    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

    fig = optuna.visualization.plot_timeline(study)
    fig.show()

[I 2023-12-08 13:01:24,379] A new study created in memory with name: no-name-0ef6e26a-cecc-469b-bf58-e040d9252c83
[I 2023-12-08 13:02:16,761] Trial 7 finished with value: 0.011940501246542388 and parameters: {'num_leaves': 26, 'learning_rate': 0.010883993998447696, 'n_estimators': 158}. Best is trial 7 with value: 0.011940501246542388.
[I 2023-12-08 13:02:26,874] Trial 0 finished with value: 0.00701096651099078 and parameters: {'num_leaves': 46, 'learning_rate': 0.030365417919529934, 'n_estimators': 126}. Best is trial 0 with value: 0.00701096651099078.
[I 2023-12-08 13:02:55,861] Trial 11 finished with value: 0.014082416459596151 and parameters: {'num_leaves': 33, 'learning_rate': 0.005081833747217318, 'n_estimators': 298}. Best is trial 0 with value: 0.00701096651099078.
[I 2023-12-08 13:03:11,521] Trial 13 finished with value: 0.009213282123563206 and parameters: {'num_leaves': 6, 'learning_rate': 0.02334264928924797, 'n_estimators': 735}. Best is trial 0 with value: 0.0070109665109

Melhores Hiperparâmetros: {'num_leaves': 27, 'learning_rate': 0.01523898302249523, 'n_estimators': 235, 'objective': 'regression', 'metric': 'rmse', 'random_state': 42, 'force_col_wise': True, 'device': 'gpu', 'gpu_platform_id': 1, 'gpu_device_id': 0}


C:\Users\jean_\AppData\Local\Temp\ipykernel_16612\2352747291.py:60: ExperimentalWarning:

plot_timeline is experimental (supported from v3.2.0). The interface can change in the future.



In [12]:
if TRAIN == True:
    # Carrega os melhores hiperparâmetros do modelo a partir do arquivo JSON
    with open(f"{model_path}lgbm_hyperparameters.json", "r") as f:
        best_params = json.load(f)

    # Imprimir os melhores hiperparâmetros
    print("Hiperparâmetros Carregados:")
    for key, value in best_params.items(): print(f"{key}: {value}")

    # Use os melhores hiperparâmetros para treinar o modelo final
    best_lgb_model = LGBMRegressor(**best_params)
    best_lgb_model.fit(X_train_lgb, y_train.ravel())

    # Salvar o modelo treinado
    best_lgb_model.booster_.save_model(f"{model_path}lgbm_model.txt")

Hiperparâmetros Carregados:
num_leaves: 27
learning_rate: 0.01523898302249523
n_estimators: 235
objective: regression
metric: rmse
random_state: 42
force_col_wise: True
device: gpu
gpu_platform_id: 1
gpu_device_id: 0
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 61200
[LightGBM] [Info] Number of data points in the train set: 69264, number of used features: 240
[LightGBM] [Info] Using requested OpenCL platform 1 device 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce GTX 1050 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 240 dense feature groups (15.85 MB) transferred to GPU in 0.012414 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 0.049724


In [13]:
# Carregar o modelo
lgb_model = lgb.Booster(model_file=f"{model_path}lgbm_model.txt")

# Geração de previsões usando o modelo carregado
forecast = lgb_model.predict(X_test_lgb)

# Reformulação das previsões para o formato adequado e inversão da normalização
forecast = forecast.reshape(-1, 1)
lgb_pred = scaler_y.inverse_transform(forecast)

In [14]:
# Exibição das métricas de avaliação e do gráfico de comparação
resultado = metric_display(y_test_inv, lgb_pred, df_final.index[valid_end_idx+past_history], df_final.index[test_end_idx-1])

# Adicionando as métricas de avaliação ao DataFrame, se o modelo já estiver no DataFrame, atualize as métricas, caso contrário, adicione uma nova linha
model_name = 'LightGBM'
if model_name in model_metrics['MODEL'].values:
    model_metrics.loc[model_metrics['MODEL'] == model_name] = [model_name] + resultado
else:
    model_metrics.loc[len(model_metrics)] = [model_name] + resultado

[Menor é melhor] RMSE: 10.5790
[Menor é melhor] MAE: 7.8871
[Menor é melhor] MAPE: 5.3409%
[Menor é melhor] SMAPE: 5.3941% 
[Maior é melhor] R²: 0.8445 
[Maior é melhor] EVS: 0.8584 


# XGBOOST

In [20]:
# Reformulação dos conjuntos de treinamento, validação e teste para o formato adequado ao XGBoost
X_train_xgb = X_train.reshape(-1, X_train.shape[1] * X_train.shape[2])
X_val_xgb = X_val.reshape(-1, X_val.shape[1] * X_val.shape[2])
X_test_xgb = X_test.reshape(-1, X_test.shape[1] * X_test.shape[2])

In [21]:
if TRAIN == True:
    # Defina a função de objetivo para otimização do Optuna
    def objective(trial):
        param = {
            'objective': 'reg:squarederror',  # Para regressão
            'eval_metric': 'rmse',
            'n_jobs': -1,
            'eta': trial.suggest_loguniform('eta', 0.001, 0.1),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
            'alpha': trial.suggest_loguniform('alpha', 1e-5, 10.0),
            'lambda': trial.suggest_loguniform('lambda', 1e-5, 10.0),
            'gamma': trial.suggest_loguniform('gamma', 1e-5, 10.0),
            'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-5, 10.0)
        }
        
        # Criação de matrizes DMatrix para os conjuntos de treinamento, validação e teste do XGBoost
        dtrain = xgb.DMatrix(X_train_xgb, y_train)
        dval = xgb.DMatrix(X_val_xgb, y_val)
        
        # Lista de avaliação para acompanhar o desempenho do modelo XGBoost durante o treinamento
        evals = [(dtrain, 'train'), (dval, 'eval')]
  
        # Treinamento do modelo XGBoost com os parâmetros definidos
        model = xgb.train(param, dtrain, num_boost_round=1000, evals=evals, early_stopping_rounds=10, verbose_eval=False)

        # Geração de previsões usando o modelo XGBoost treinado
        forecast = model.predict(xgb.DMatrix(X_test_xgb))

        # Cálculo do erro quadrático médio
        rmse = np.sqrt(mean_squared_error(y_test, forecast))

        return rmse

    # Crie um estudo Optuna e otimize a função objetivo
    study = optuna.create_study(direction="minimize")  # Minimize o RMSE
    study.optimize(objective, n_trials=n_trials, n_jobs=-1)  # Execute n_trials tentativas de otimização

    # Obtenha os melhores hiperparâmetros encontrados
    best_params = study.best_params

    # Adicionando os parâmetros fixos, que não foram otimizados pelo Optuna
    best_params.update({
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'n_jobs': -1,
    })

    # Salvando os hiperparâmetros do modelo em um arquivo JSON
    with open(f"{model_path}xgboost_hyperparameters.json", "w") as f:
        json.dump(best_params, f)

    # Imprimir os melhores hiperparâmetros
    print("Melhores Hiperparâmetros:", best_params)

    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

    fig = optuna.visualization.plot_timeline(study)
    fig.show()

In [22]:
if TRAIN == True:
    # Carrega os melhores hiperparâmetros do modelo a partir do arquivo JSON
    with open(f"{model_path}xgboost_hyperparameters.json", "r") as f:
        best_params = json.load(f)

    # Imprimir os melhores hiperparâmetros
    print("Hiperparâmetros Carregados:")
    for key, value in best_params.items(): print(f"{key}: {value}")

    # Use os melhores hiperparâmetros para treinar o modelo final
    best_xgb_model = xgb.train(best_params, xgb.DMatrix(X_train_xgb, y_train), num_boost_round=1000)

    # Salvar o modelo treinado
    best_xgb_model.save_model(f"{model_path}xgboost_model.model")

In [23]:
# Carregar o modelo salvo
loaded_xgb_model = xgb.Booster(model_file=f"{model_path}xgboost_model.model")

# Geração de previsões usando o modelo carregado
forecast = loaded_xgb_model.predict(xgb.DMatrix(X_test_xgb))

# Reformulação das previsões para o formato adequado e inversão da normalização
forecast = forecast.reshape(-1, 1)
xgb_pred = scaler_y.inverse_transform(forecast)

In [24]:
# Exibição das métricas de avaliação e do gráfico de comparação
resultado = metric_display(y_test_inv, xgb_pred, df_final.index[valid_end_idx+past_history], df_final.index[test_end_idx-1])

# Adicionando as métricas de avaliação ao DataFrame, se o modelo já estiver no DataFrame, atualize as métricas, caso contrário, adicione uma nova linha
model_name = 'XGBoost'
if model_name in model_metrics['MODEL'].values:
    model_metrics.loc[model_metrics['MODEL'] == model_name] = [model_name] + resultado
else:
    model_metrics.loc[len(model_metrics)] = [model_name] + resultado

[Menor é melhor] RMSE: 259.6598
[Menor é melhor] MAE: 180.3287
[Menor é melhor] MAPE: inf%
[Menor é melhor] SMAPE: 13.7573% 
[Maior é melhor] R²: 0.7270 
[Maior é melhor] EVS: 0.7344 


# LSTM

In [20]:
# Limpeza do ambiente do TensorFlow
clear_session()

if TUNE == True:
    def objective(trial):
        # Define os hiperparâmetros que deseja otimizar
        num_units = trial.suggest_int("num_units", 50, 200)
        dropout_rate = trial.suggest_uniform("dropout_rate", 0.0, 0.3)
        learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)

        # Crie o modelo com os hiperparâmetros sugeridos
        multivariate_lstm = Sequential([
            LSTM(num_units, input_shape=input_shape, return_sequences=True),
            Flatten(),
            Dense(2 * num_units, activation='relu'),
            Dropout(dropout_rate),
            Dense(1)
        ])

        # Compile o modelo com a taxa de aprendizado sugerida
        optimizer = Adam(learning_rate=learning_rate, amsgrad=True)
        multivariate_lstm.compile(loss=loss, optimizer=optimizer, metrics=metric)

        # Treine o modelo
        multivariate_lstm.fit(train, epochs=120, validation_data=validation, callbacks=[early_stopping], verbose=0)

        # Avalie o modelo e retorne a métrica que deseja otimizar (perda no final)
        y_pred = multivariate_lstm.predict(X_test)
        y_pred_inv = scaler_y.inverse_transform(y_pred)
        rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))

        return rmse

    # Crie um estudo Optuna com o pruner
    study = optuna.create_study(direction="minimize")   # Minimize o RMSE
    study.optimize(objective, n_trials=n_trials, n_jobs=-1)   # Execute n_trials tentativas de otimização

    # Obtenha os melhores parâmetros
    best_params = study.best_params
    best_loss = study.best_value

    # Salvando os hiperparâmetros do modelo em um arquivo JSON
    with open(f"{model_path}multivariate_lstm_hyperparameters.json", "w") as f:
        json.dump(best_params, f)

    # Imprimir os melhores hiperparâmetros
    print("Melhores Hiperparâmetros:", best_params)

    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

    fig = optuna.visualization.plot_timeline(study)
    fig.show()

[I 2023-11-10 23:17:23,424] A new study created in memory with name: no-name-821860cd-04fe-41ea-8bd7-c593251b55e9


20/20 [==============================] - 3s 42ms/step


[I 2023-11-10 23:44:06,866] Trial 4 finished with value: 52.9212728229474 and parameters: {'num_units': 68, 'dropout_rate': 0.21747744377030567, 'learning_rate': 0.00017325071921395872}. Best is trial 4 with value: 52.9212728229474.


20/20 [==============================] - 3s 38ms/step


[I 2023-11-10 23:51:57,157] Trial 12 finished with value: 43.279834954386445 and parameters: {'num_units': 53, 'dropout_rate': 0.07107757016029785, 'learning_rate': 8.718117031351212e-05}. Best is trial 12 with value: 43.279834954386445.


...

[I 2023-11-12 09:59:59,627] Trial 8 finished with value: 21.46727422931969 and parameters: {'num_units': 191, 'dropout_rate': 0.17226201075159067, 'learning_rate': 0.000639264431798566}. Best is trial 7 with value: 14.982181738964723.


20/20 [==============================] - 2s 59ms/step


[I 2023-11-12 10:20:00,909] Trial 2 finished with value: 22.154944762339607 and parameters: {'num_units': 197, 'dropout_rate': 0.2042435275093921, 'learning_rate': 3.2179655451393713e-05}. Best is trial 7 with value: 14.982181738964723.


Melhores Hiperparâmetros: {'num_units': 108, 'dropout_rate': 0.08937065134096495, 'learning_rate': 0.0008057456454138645}


C:\Users\jean_\AppData\Local\Temp\ipykernel_15924\3168578168.py:52: ExperimentalWarning:

plot_timeline is experimental (supported from v3.2.0). The interface can change in the future.



In [21]:
if TRAIN == True:
    # Carrega os melhores hiperparâmetros do modelo a partir do arquivo JSON
    with open(f"{model_path}multivariate_lstm_hyperparameters.json", "r") as f:
        best_params = json.load(f)

    # Imprimir os melhores hiperparâmetros
    print("Hiperparâmetros Carregados:")
    for key, value in best_params.items(): print(f"{key}: {value}")
    
    # Use os melhores parâmetros para treinar o modelo final
    best_num_units = best_params["num_units"]
    best_dropout_rate = best_params["dropout_rate"]
    best_learning_rate = best_params["learning_rate"]

    # Definição do modelo com os melhores parâmetros
    multivariate_lstm = Sequential([
        LSTM(best_num_units, input_shape=input_shape, return_sequences=True),
        Flatten(),
        Dense(2 * best_num_units, activation='relu'),
        Dropout(best_dropout_rate),
        Dense(1)
    ])

    # Definição de Callbacks
    model_checkpoint = ModelCheckpoint(f'{model_path}multivariate_lstm.h5', monitor=('val_loss'), save_best_only=True)
    optimizer = Adam(learning_rate=best_learning_rate, amsgrad=True)

    # Compilação do modelo
    multivariate_lstm.compile(loss=loss, optimizer=optimizer, metrics=metric)

    # Treinamento do modelo
    history = multivariate_lstm.fit(train, epochs=120, validation_data=validation, callbacks=[early_stopping, model_checkpoint])

    # Plotagem da história de treinamento
    plot_training_history(history)

Hiperparâmetros Carregados:
num_units: 108
dropout_rate: 0.08937065134096495
learning_rate: 0.0008057456454138645
Epoch 1/120
2165/2165 [==============================] - 204s 93ms/step - loss: 5.2537e-04 - root_mean_squared_error: 0.0228 - val_loss: 3.9018e-04 - val_root_mean_squared_error: 0.0198
Epoch 2/120
2165/2165 [==============================] - 202s 93ms/step - loss: 8.7249e-05 - root_mean_squared_error: 0.0093 - val_loss: 4.9947e-04 - val_root_mean_squared_error: 0.0223
Epoch 3/120
2165/2165 [==============================] - 200s 92ms/step - loss: 8.0272e-05 - root_mean_squared_error: 0.0090 - val_loss: 3.0158e-04 - val_root_mean_squared_error: 0.0174
Epoch 4/120
2165/2165 [==============================] - 199s 92ms/step - loss: 7.5436e-05 - root_mean_squared_error: 0.0087 - val_loss: 3.0125e-04 - val_root_mean_squared_error: 0.0174
Epoch 5/120
2165/2165 [==============================] - 200s 92ms/step - loss: 6.4388e-05 - root_mean_squared_error: 0.0080 - val_loss: 2.367

In [22]:
# Carregamento do modelo com melhor desempenho
multivariate_lstm = load_model(f'{model_path}multivariate_lstm.h5')

# Geração de previsões usando o modelo treinado
forecast = multivariate_lstm.predict(X_test)
lstm_pred = scaler_y.inverse_transform(forecast)

20/20 [==============================] - 1s 25ms/step


In [23]:
# Exibição das métricas de avaliação e do gráfico de comparação
resultado = metric_display(y_test_inv, lstm_pred, df_final.index[valid_end_idx+past_history], df_final.index[test_end_idx-1])

# Adicionando as métricas de avaliação ao DataFrame, se o modelo já estiver no DataFrame, atualize as métricas, caso contrário, adicione uma nova linha
model_name = 'LSTM'
if model_name in model_metrics['MODEL'].values:
    model_metrics.loc[model_metrics['MODEL'] == model_name] = [model_name] + resultado
else:
    model_metrics.loc[len(model_metrics)] = [model_name] + resultado

[Menor é melhor] RMSE: 16.7370
[Menor é melhor] MAE: 13.1297
[Menor é melhor] MAPE: 9.0692%
[Menor é melhor] SMAPE: 9.3427% 
[Maior é melhor] R²: 0.6108 
[Maior é melhor] EVS: 0.6164 


# STACKED LSTM

In [24]:
# Limpeza do ambiente do TensorFlow
clear_session()

if TUNE == True:
    def objective(trial):
        # Define os hiperparâmetros que deseja otimizar
        num_units = trial.suggest_int("num_units", 25, 125)
        dropout_rate = trial.suggest_uniform("dropout_rate", 0.0, 0.3)
        learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)

        multivariate_stacked_lstm = Sequential([
            LSTM(2 * num_units, input_shape=input_shape, return_sequences=True),
            LSTM(num_units, return_sequences=True),
            Flatten(),
            Dense(num_units, activation='relu'),
            Dropout(dropout_rate),
            Dense(1)
        ])

        # Compile o modelo com a taxa de aprendizado sugerida
        optimizer = Adam(learning_rate=learning_rate, amsgrad=True)
        multivariate_stacked_lstm.compile(loss=loss, optimizer=optimizer, metrics=metric)

        # Treine o modelo
        multivariate_stacked_lstm.fit(train, epochs=120, validation_data=validation, callbacks=[early_stopping], verbose=0)

        # Avalie o modelo e retorne a métrica que deseja otimizar (perda no final)
        y_pred = multivariate_stacked_lstm.predict(X_test)
        y_pred_inv = scaler_y.inverse_transform(y_pred)
        rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))

        return rmse

    # Crie um estudo Optuna
    study = optuna.create_study(direction="minimize")  # Minimize o RMSE
    study.optimize(objective, n_trials=n_trials)  # Execute n_trials tentativas de otimização com 10 minutos de timeout

    # Obtenha os melhores parâmetros
    best_params = study.best_params
    best_loss = study.best_value

    # Salvando os hiperparâmetros do modelo em um arquivo JSON
    with open(f"{model_path}multivariate_stacked_lstm_hyperparameters.json", "w") as f:
        json.dump(best_params, f)

    # Imprimir os melhores hiperparâmetros
    print("Melhores Hiperparâmetros:", best_params)

    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

    fig = optuna.visualization.plot_timeline(study)
    fig.show()

[I 2023-11-12 13:24:18,188] A new study created in memory with name: no-name-9c53f5cf-c884-4f41-9548-7251d234de05


20/20 [==============================] - 3s 106ms/step


[I 2023-11-13 06:38:42,576] Trial 0 finished with value: 23.087929358293383 and parameters: {'num_units': 109, 'dropout_rate': 0.014532264228807977, 'learning_rate': 0.00032633234891857115}. Best is trial 0 with value: 23.087929358293383.


20/20 [==============================] - 2s 59ms/step


[I 2023-11-13 11:20:33,115] Trial 1 finished with value: 24.047594994686236 and parameters: {'num_units': 67, 'dropout_rate': 0.23988254267295914, 'learning_rate': 0.0011974504592871115}. Best is trial 0 with value: 23.087929358293383.
[W 2023-11-13 14:05:42,190] Trial 2 failed with parameters: {'num_units': 48, 'dropout_rate': 0.0360752562055469, 'learning_rate': 0.0004382018167708845} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "d:\Github\TCC\TCC_env\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\jean_\AppData\Local\Temp\ipykernel_15924\1074901656.py", line 25, in objective
    multivariate_stacked_lstm.fit(train, epochs=120, validation_data=validation, callbacks=[early_stopping], verbose=0)
  File "d:\Github\TCC\TCC_env\lib\site-packages\keras\src\utils\traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "d:\Github\TCC\TCC_env\

KeyboardInterrupt: 

In [ ]:
if TRAIN == True:
    # Carrega os melhores hiperparâmetros do modelo a partir do arquivo JSON
    with open(f"{model_path}multivariate_stacked_lstm_hyperparameters.json", "r") as f:
        best_params = json.load(f)

    # Imprimir os melhores hiperparâmetros
    print("Hiperparâmetros Carregados:")
    for key, value in best_params.items(): print(f"{key}: {value}")

    # Use os melhores parâmetros para treinar o modelo final
    best_num_units = best_params["num_units"]
    best_dropout_rate = best_params["dropout_rate"]
    best_learning_rate = best_params["learning_rate"]

    # Definição do modelo com os melhores parâmetros
    multivariate_stacked_lstm = Sequential([
        LSTM(2 * best_num_units, input_shape=input_shape, return_sequences=True),
        LSTM(best_num_units, return_sequences=True),
        Flatten(),
        Dense(best_num_units, activation='relu'),
        Dropout(best_dropout_rate),
        Dense(1)
    ])

    # Definição de Callbacks
    model_checkpoint = ModelCheckpoint(f'{model_path}multivariate_stacked_lstm.h5', monitor=('val_loss'), save_best_only=True)
    optimizer = Adam(learning_rate=best_learning_rate, amsgrad=True)

    # Compilação do modelo
    multivariate_stacked_lstm.compile(loss=loss, optimizer=optimizer, metrics=metric)

    # Treinamento do modelo
    history = multivariate_stacked_lstm.fit(train, epochs=120, validation_data=validation, callbacks=[early_stopping, model_checkpoint])

    # Plotagem da história de treinamento
    plot_training_history(history)

In [ ]:
# Carregamento do modelo com melhor desempenho
multivariate_stacked_lstm = load_model(f'{model_path}multivariate_stacked_lstm.h5')

# Geração de previsões usando o modelo treinado
forecast = multivariate_stacked_lstm.predict(X_test)
multivariate_stacked_lstm_pred = scaler_y.inverse_transform(forecast)

20/20 [==============================] - 1s 13ms/step


In [ ]:
# Exibição das métricas de avaliação e do gráfico de comparação
resultado = metric_display(y_test_inv, multivariate_stacked_lstm_pred, df_final.index[valid_end_idx+past_history], df_final.index[test_end_idx-1])

# Adicionando as métricas de avaliação ao DataFrame, se o modelo já estiver no DataFrame, atualize as métricas, caso contrário, adicione uma nova linha
model_name = 'Stacked LSTM'
if model_name in model_metrics['MODEL'].values:
    model_metrics.loc[model_metrics['MODEL'] == model_name] = [model_name] + resultado
else:
    model_metrics.loc[len(model_metrics)] = [model_name] + resultado

[Menor é melhor] RMSE: 257.7370
[Menor é melhor] MAE: 178.2124
[Menor é melhor] MAPE: inf%
[Menor é melhor] SMAPE: 14.1049% 
[Maior é melhor] R²: 0.7310 
[Maior é melhor] EVS: 0.7631 


# CNN

In [ ]:
# Limpeza do ambiente do TensorFlow
clear_session()

if TUNE == True:
    def objective(trial):
        # Define os hiperparâmetros que deseja otimizar
        num_filters = trial.suggest_int("num_filters", 32, 128)
        kernel_size = trial.suggest_int("kernel_size", 2, 5)
        num_dense_units = trial.suggest_int("num_dense_units", 32, 64)
        learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)

        multivariate_cnn = Sequential([
            Conv1D(filters=num_filters, kernel_size=kernel_size, strides=1, padding='causal', activation='relu', input_shape=input_shape),
            Flatten(),
            Dense(num_dense_units, activation='relu'),
            Dense(1)
        ])

        # Compile o modelo com a taxa de aprendizado sugerida
        optimizer = Adam(learning_rate=learning_rate, amsgrad=True)
        multivariate_cnn.compile(loss=loss, optimizer=optimizer, metrics=metric)

        # Treine o modelo
        multivariate_cnn.fit(train, epochs=120, validation_data=validation, callbacks=[early_stopping], verbose=0)

        # Avalie o modelo e retorne a métrica que deseja otimizar (perda no final)
        y_pred = multivariate_cnn.predict(X_test)
        y_pred_inv = scaler_y.inverse_transform(y_pred)
        rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))

        return rmse

    # Crie um estudo Optuna
    study = optuna.create_study(direction="minimize")  # Minimize o RMSE
    study.optimize(objective, n_trials=n_trials)  # Execute n_trials tentativas de otimização com 10 minutos de timeout

    # Obtenha os melhores parâmetros
    best_params = study.best_params
    best_loss = study.best_value

    # Salvando os hiperparâmetros do modelo em um arquivo JSON
    with open(f"{model_path}multivariate_cnn_hyperparameters.json", "w") as f:
        json.dump(best_params, f)

    # Imprimir os melhores hiperparâmetros
    print("Melhores Hiperparâmetros:", best_params)

    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

    fig = optuna.visualization.plot_timeline(study)
    fig.show()

In [ ]:
if TRAIN == True:
    # Carrega os melhores hiperparâmetros do modelo a partir do arquivo JSON
    with open(f"{model_path}multivariate_cnn_hyperparameters.json", "r") as f:
        best_params = json.load(f)

    # Imprimir os melhores hiperparâmetros
    print("Hiperparâmetros Carregados:")
    for key, value in best_params.items(): print(f"{key}: {value}")

    # Use os melhores parâmetros para treinar o modelo final
    best_num_filters = best_params["num_filters"]
    best_kernel_size = best_params["kernel_size"]
    best_num_dense_units = best_params["num_dense_units"]
    best_learning_rate = best_params["learning_rate"]

    # Definição do modelo com os melhores parâmetros
    multivariate_cnn = Sequential([
        Conv1D(filters=best_num_units, kernel_size=best_kernel_size, strides=1, padding='causal', activation='relu', input_shape=input_shape),
        Flatten(),
        Dense(best_num_dense_units, activation='relu'),
        Dense(1)
    ])

    # Definição de Callbacks
    model_checkpoint = ModelCheckpoint(f'{model_path}multivariate_cnn.h5', save_best_only=True)
    optimizer = Adam(learning_rate=best_learning_rate, amsgrad=True)

    # Compilação do modelo
    multivariate_cnn.compile(loss=loss, optimizer=optimizer, metrics=metric)

    # Treinamento do modelo
    history = multivariate_cnn.fit(train, epochs=120, validation_data=validation, callbacks=[early_stopping, model_checkpoint])

    # Plotagem da história de treinamento
    plot_training_history(history)

In [ ]:
# Carregamento do modelo com melhor desempenho
multivariate_cnn = load_model(f'{model_path}multivariate_cnn.h5')

# Geração de previsões usando o modelo treinado
forecast = multivariate_cnn.predict(X_test)
multivariate_cnn_pred = scaler_y.inverse_transform(forecast)

20/20 [==============================] - 0s 1ms/step


In [ ]:
# Exibição das métricas de avaliação e do gráfico de comparação
metric_display(y_test_inv, multivariate_cnn_pred, df_final.index[valid_end_idx+past_history], df_final.index[test_end_idx-1])

# Adicionando as métricas de avaliação ao DataFrame, se o modelo já estiver no DataFrame, atualize as métricas, caso contrário, adicione uma nova linha
model_name = 'CNN'
if model_name in model_metrics['MODEL'].values:
    model_metrics.loc[model_metrics['MODEL'] == model_name] = [model_name] + resultado
else:
    model_metrics.loc[len(model_metrics)] = [model_name] + resultado

[Menor é melhor] RMSE: 279.6872
[Menor é melhor] MAE: 198.3150
[Menor é melhor] MAPE: inf%
[Menor é melhor] SMAPE: 14.4816% 
[Maior é melhor] R²: 0.6833 
[Maior é melhor] EVS: 0.6879 


# CNN-LSTM

In [10]:
# Limpeza do ambiente do TensorFlow
clear_session()

if TUNE == True:
    def objective(trial):
        # Define os hiperparâmetros que deseja otimizar
        num_filters = trial.suggest_int("num_filters", 32, 128)
        kernel_size = trial.suggest_int("kernel_size", 2, 5)
        num_units = trial.suggest_int("num_units", 50, 200)
        dropout_rate = trial.suggest_uniform("dropout_rate", 0.0, 0.3)
        learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)

        multivariate_cnn_lstm = Sequential([
            Conv1D(filters=num_filters, kernel_size=kernel_size,
                strides=1, padding='causal',
                activation='relu', 
                input_shape=input_shape),
            LSTM(num_units, return_sequences=True),
            Flatten(),
            Dense(2*num_units, activation='relu'),
            Dropout(dropout_rate),
            Dense(1)
        ])

        # Compile o modelo com a taxa de aprendizado sugerida
        optimizer = Adam(learning_rate=learning_rate, amsgrad=True)
        multivariate_cnn_lstm.compile(loss=loss, optimizer=optimizer, metrics=metric)

        # Treine o modelo
        multivariate_cnn_lstm.fit(train, epochs=120, validation_data=validation, callbacks=[early_stopping], verbose=0)

        # Avalie o modelo e retorne a métrica que deseja otimizar (perda no final)
        y_pred = multivariate_cnn_lstm.predict(X_test)
        y_pred_inv = scaler_y.inverse_transform(y_pred)
        rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))

        return rmse

    # Crie um estudo Optuna
    study = optuna.create_study(direction="minimize")  # Minimize o RMSE
    study.optimize(objective, n_trials=n_trials)  # Execute n_trials tentativas de otimização com 10 minutos de timeout

    # Obtenha os melhores parâmetros
    best_params = study.best_params
    best_loss = study.best_value

    # Salvando os hiperparâmetros do modelo em um arquivo JSON
    with open(f"{model_path}multivariate_cnn_lstm_hyperparameters.json", "w") as f:
        json.dump(best_params, f)

    # Imprimir os melhores hiperparâmetros
    print("Melhores Hiperparâmetros:", best_params)

    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

    fig = optuna.visualization.plot_timeline(study)
    fig.show()

[I 2023-11-13 21:40:59,633] A new study created in memory with name: no-name-23e5ee6b-8551-4763-a913-8db427cd047c


20/20 [==============================] - 1s 10ms/step


[I 2023-11-13 22:39:39,012] Trial 0 finished with value: 32.4184106327549 and parameters: {'num_filters': 77, 'kernel_size': 2, 'num_units': 119, 'dropout_rate': 0.18873646307515582, 'learning_rate': 0.00011505599039087604}. Best is trial 0 with value: 32.4184106327549.


20/20 [==============================] - 1s 12ms/step


[I 2023-11-14 00:44:18,863] Trial 1 finished with value: 20.851396794955754 and parameters: {'num_filters': 112, 'kernel_size': 2, 'num_units': 147, 'dropout_rate': 0.13251872460408604, 'learning_rate': 0.0008601309120322559}. Best is trial 1 with value: 20.851396794955754.


...

[I 2023-11-14 08:22:37,199] Trial 8 finished with value: 17.892025081139714 and parameters: {'num_filters': 73, 'kernel_size': 4, 'num_units': 141, 'dropout_rate': 0.16079616500853527, 'learning_rate': 0.005192875582575651}. Best is trial 6 with value: 17.440831397452975.


20/20 [==============================] - 1s 8ms/step


[I 2023-11-14 08:54:54,946] Trial 9 finished with value: 20.46092004424496 and parameters: {'num_filters': 80, 'kernel_size': 3, 'num_units': 73, 'dropout_rate': 0.08889471070313047, 'learning_rate': 0.003845181008108681}. Best is trial 6 with value: 17.440831397452975.


20/20 [==============================] - 1s 17ms/step


[I 2023-11-14 09:52:02,437] Trial 10 finished with value: 32.556637795185146 and parameters: {'num_filters': 99, 'kernel_size': 4, 'num_units': 192, 'dropout_rate': 0.02293603445612069, 'learning_rate': 1.3514981277444402e-05}. Best is trial 6 with value: 17.440831397452975.


In [ ]:
# Limpeza do ambiente do TensorFlow
clear_session()

if TRAIN == True:
    # Carrega os melhores hiperparâmetros do modelo a partir do arquivo JSON
    with open(f"{model_path}multivariate_cnn_hyperparameters.json", "r") as f:
        best_params = json.load(f)

    # Imprimir os melhores hiperparâmetros
    print("Hiperparâmetros Carregados:")
    for key, value in best_params.items(): print(f"{key}: {value}")

    # Use os melhores parâmetros para treinar o modelo final
    best_num_filters = best_params["num_filters"]
    best_kernel_size = best_params["kernel_size"]
    best_num_units = best_params["num_units"]
    best_dropout_rate = best_params["dropout_rate"]
    best_learning_rate = best_params["learning_rate"]
    
    # Definição do modelo
    multivariate_cnn_lstm = Sequential([
        Conv1D(filters=best_num_filters, kernel_size=best_kernel_size,
            strides=1, padding='causal',
            activation='relu', 
            input_shape=input_shape),
        LSTM(best_num_units, return_sequences=True),
        Flatten(),
        Dense(2*best_num_units, activation='relu'),
        Dropout(best_dropout_rate),
        Dense(1)
    ])

    # Definição de Callbacks
    model_checkpoint = ModelCheckpoint(f'{model_path}multivariate_cnn_lstm.h5', save_best_only=True)
    optimizer = Adam(learning_rate=best_learning_rate, amsgrad=True)

    # Compilação do modelo
    multivariate_cnn_lstm.compile(loss=loss, optimizer=optimizer, metrics=metric)

    # Treinamento do modelo
    history = multivariate_cnn_lstm.fit(train, epochs=120, validation_data=validation, callbacks=[early_stopping, model_checkpoint])

    # Plotagem da história de treinamento
    plot_training_history(history)

In [ ]:
# Carregamento do modelo com melhor desempenho
multivariate_cnn_lstm = load_model(f'{model_path}multivariate_cnn_lstm.h5')

# Geração de previsões usando o modelo treinado
forecast = multivariate_cnn_lstm.predict(X_test)
multivariate_cnn_lstm_pred = scaler_y.inverse_transform(forecast)

20/20 [==============================] - 1s 6ms/step


In [ ]:
# Exibição das métricas de avaliação e do gráfico de comparação
resultado = metric_display(y_test_inv, multivariate_cnn_lstm_pred, df_final.index[valid_end_idx+past_history], df_final.index[test_end_idx-1])

# Adicionando as métricas de avaliação ao DataFrame, se o modelo já estiver no DataFrame, atualize as métricas, caso contrário, adicione uma nova linha
model_name = 'CNN-LSTM'
if model_name in model_metrics['MODEL'].values:
    model_metrics.loc[model_metrics['MODEL'] == model_name] = [model_name] + resultado
else:
    model_metrics.loc[len(model_metrics)] = [model_name] + resultado

[Menor é melhor] RMSE: 266.2127
[Menor é melhor] MAE: 197.2141
[Menor é melhor] MAPE: inf%
[Menor é melhor] SMAPE: 14.8254% 
[Maior é melhor] R²: 0.7131 
[Maior é melhor] EVS: 0.7160 


# TS MLP

In [ ]:
# Limpeza do ambiente do TensorFlow
clear_session()

if TUNE == True:
    def objective(trial):
        # Define os hiperparâmetros que deseja otimizar
        num_units_layer_0 = trial.suggest_int("num_units_layer_0", 50, 200)
        num_units_layer_1 = trial.suggest_int("num_units_layer_1", 50, 200)
        num_units_layer_2 = trial.suggest_int("num_units_layer_2", 50, 200)
        num_units_layer_3 = trial.suggest_int("num_units_layer_3", 50, 200)
        num_units_layer_4 = trial.suggest_int("num_units_layer_4", 50, 200)
        dropout_rate = trial.suggest_uniform("dropout_rate", 0.0, 0.3)
        learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)

        multivariate_mlp = tf.keras.models.Sequential([
            TimeDistributed(Dense(num_units_layer_0, activation='relu'), input_shape=input_shape),
            TimeDistributed(Dense(num_units_layer_1, activation='relu')),
            TimeDistributed(Dense(num_units_layer_2, activation='relu')),
            TimeDistributed(Dense(num_units_layer_3, activation='relu')),
            Flatten(),
            Dense(num_units_layer_4, activation='relu'),
            Dropout(dropout_rate),
            Dense(1)
        ])

        # Compile o modelo com a taxa de aprendizado sugerida
        optimizer = Adam(learning_rate=learning_rate, amsgrad=True)
        multivariate_mlp.compile(loss=loss, optimizer=optimizer, metrics=metric)

        # Treine o modelo
        multivariate_mlp.fit(train, epochs=120, validation_data=validation, callbacks=[early_stopping], verbose=0)

        # Avalie o modelo e retorne a métrica que deseja otimizar (perda no final)
        y_pred = multivariate_mlp.predict(X_test)
        y_pred_inv = scaler_y.inverse_transform(y_pred)
        rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))

        return rmse

    # Crie um estudo Optuna
    study = optuna.create_study(direction="minimize")  # Minimize o RMSE
    study.optimize(objective, n_trials=n_trials)  # Execute n_trials tentativas de otimização com 10 minutos de timeout

    # Obtenha os melhores parâmetros
    best_params = study.best_params
    best_loss = study.best_value

    # Salvando os hiperparâmetros do modelo em um arquivo JSON
    with open(f"{model_path}multivariate_mlp_hyperparameters.json", "w") as f:
        json.dump(best_params, f)

    # Imprimir os melhores hiperparâmetros
    print("Melhores Hiperparâmetros:", best_params)

    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

    fig = optuna.visualization.plot_timeline(study)
    fig.show()

In [ ]:
# Limpeza do ambiente do TensorFlow
clear_session()

if TRAIN == True:
    # Carrega os melhores hiperparâmetros do modelo a partir do arquivo JSON
    with open(f"{model_path}multivariate_mlp_hyperparameters.json", "r") as f:
        best_params = json.load(f)

    # Imprimir os melhores hiperparâmetros
    print("Hiperparâmetros Carregados:")
    for key, value in best_params.items(): print(f"{key}: {value}")

    # Use os melhores parâmetros para treinar o modelo final
    best_num_units_layer_0 = best_params["num_units_layer_0"]
    best_num_units_layer_1 = best_params["num_units_layer_1"]
    best_num_units_layer_2 = best_params["num_units_layer_2"]
    best_num_units_layer_3 = best_params["num_units_layer_3"]
    best_num_units_layer_4 = best_params["num_units_layer_4"]
    best_dropout_rate = best_params["dropout_rate"]
    best_learning_rate = best_params["learning_rate"]
    
    # Definição do modelo
    multivariate_mlp = tf.keras.models.Sequential([
        TimeDistributed(Dense(best_num_units_layer_0, activation='relu'), input_shape=input_shape),
        TimeDistributed(Dense(best_num_units_layer_1, activation='relu')),
        TimeDistributed(Dense(best_num_units_layer_2, activation='relu')),
        TimeDistributed(Dense(best_num_units_layer_3, activation='relu')),
        Flatten(),
        Dense(best_num_units_layer_4, activation='relu'),
        Dropout(best_dropout_rate),
        Dense(1)
    ])

    # Definição de Callbacks
    model_checkpoint = ModelCheckpoint(f'{model_path}multivariate_mlp.h5', save_best_only=True)
    optimizer = Adam(learning_rate=best_learning_rate, amsgrad=True)

    # Compilação do modelo
    multivariate_mlp.compile(loss=loss, optimizer=optimizer, metrics=metric)

    # Treinamento do modelo
    history = multivariate_mlp.fit(train, epochs=120, validation_data=validation, callbacks=[early_stopping, model_checkpoint])

    # Plotagem da história de treinamento
    plot_training_history(history)

In [ ]:
# Carregamento do modelo com melhor desempenho
multivariate_mlp = load_model(f'{model_path}multivariate_mlp.h5')

# Geração de previsões usando o modelo treinado
forecast = multivariate_mlp.predict(X_test)
multivariate_mlp_pred = scaler_y.inverse_transform(forecast)

20/20 [==============================] - 0s 3ms/step


In [ ]:
# Exibição das métricas de avaliação e do gráfico de comparação
resultado = metric_display(y_test_inv, multivariate_mlp_pred, df_final.index[valid_end_idx+past_history], df_final.index[test_end_idx-1])

# Adicionando as métricas de avaliação ao DataFrame, se o modelo já estiver no DataFrame, atualize as métricas, caso contrário, adicione uma nova linha
model_name = 'MLP'
if model_name in model_metrics['MODEL'].values:
    model_metrics.loc[model_metrics['MODEL'] == model_name] = [model_name] + resultado
else:
    model_metrics.loc[len(model_metrics)] = [model_name] + resultado

[Menor é melhor] RMSE: 266.6897
[Menor é melhor] MAE: 194.5958
[Menor é melhor] MAPE: inf%
[Menor é melhor] SMAPE: 14.3888% 
[Maior é melhor] R²: 0.7120 
[Maior é melhor] EVS: 0.7245 


# ENCODER-DECODER

In [ ]:
# Limpeza do ambiente do TensorFlow
clear_session()

if TUNE != True:
    def objective(trial):
        # Define os hiperparâmetros que deseja otimizar
        num_units_layer_0 = trial.suggest_int("num_units_layer_0", 50, 200)
        num_units_layer_1 = trial.suggest_int("num_units_layer_1", 50, 200)
        num_units_layer_2 = trial.suggest_int("num_units_layer_2", 50, 200)
        num_units_layer_3 = trial.suggest_int("num_units_layer_3", 50, 200)
        learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)

        encoder_decoder = Sequential([
            LSTM(num_units_layer_0, activation='relu', input_shape=input_shape),
            RepeatVector(past_history),
            LSTM(num_units_layer_1, activation='relu', return_sequences=True),
            TimeDistributed(Dense(num_units_layer_2, activation='relu')),
            Flatten(),
            Dense(num_units_layer_3, activation='relu'),
            Dense(1)
        ])

        # Compile o modelo com a taxa de aprendizado sugerida
        optimizer = Adam(learning_rate=learning_rate, amsgrad=True)
        encoder_decoder.compile(loss=loss, optimizer=optimizer, metrics=metric)

        # Treine o modelo
        encoder_decoder.fit(train, epochs=120, validation_data=validation, callbacks=[early_stopping], verbose=0)

        # Avalie o modelo e retorne a métrica que deseja otimizar (perda no final)
        y_pred = encoder_decoder.predict(X_test)
        y_pred_inv = scaler_y.inverse_transform(y_pred)
        rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))

        return rmse

    # Crie um estudo Optuna
    study = optuna.create_study(direction="minimize")  # Minimize o RMSE
    study.optimize(objective, n_trials=n_trials)  # Execute n_trials tentativas de otimização com 10 minutos de timeout

    # Obtenha os melhores parâmetros
    best_params = study.best_params
    best_loss = study.best_value

    # Salvando os hiperparâmetros do modelo em um arquivo JSON
    with open(f"{model_path}encoder_decoder_hyperparameters.json", "w") as f:
        json.dump(best_params, f)

    # Imprimir os melhores hiperparâmetros
    print("Melhores Hiperparâmetros:", best_params)

    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

    fig = optuna.visualization.plot_timeline(study)
    fig.show()

[I 2023-11-06 23:11:05,387] A new study created in memory with name: no-name-133ab751-6551-489d-9592-0f98f2d95aa3


20/20 [==============================] - 4s 156ms/step


[I 2023-11-06 23:23:03,826] Trial 6 finished with value: 345.17939837341635 and parameters: {'num_units_layer_0': 119, 'num_units_layer_1': 148, 'num_units_layer_2': 51, 'num_units_layer_3': 141, 'learning_rate': 0.0010621494403562663}. Best is trial 6 with value: 345.17939837341635.


20/20 [==============================] - 8s 384ms/step


[I 2023-11-06 23:26:27,838] Trial 12 finished with value: 917.2405736940152 and parameters: {'num_units_layer_0': 176, 'num_units_layer_1': 173, 'num_units_layer_2': 142, 'num_units_layer_3': 68, 'learning_rate': 5.168465756651407e-05}. Best is trial 6 with value: 345.17939837341635.


...

[I 2023-11-07 11:11:39,138] Trial 9 finished with value: 267.5679510716266 and parameters: {'num_units_layer_0': 186, 'num_units_layer_1': 100, 'num_units_layer_2': 178, 'num_units_layer_3': 124, 'learning_rate': 1.4915919854415123e-05}. Best is trial 0 with value: 258.61864429107897.


20/20 [==============================] - 4s 158ms/step


[I 2023-11-07 11:20:21,774] Trial 1 finished with value: 272.3502135199492 and parameters: {'num_units_layer_0': 185, 'num_units_layer_1': 113, 'num_units_layer_2': 142, 'num_units_layer_3': 174, 'learning_rate': 0.00013229882252863227}. Best is trial 0 with value: 258.61864429107897.


Melhores Hiperparâmetros: {'num_units_layer_0': 162, 'num_units_layer_1': 103, 'num_units_layer_2': 113, 'num_units_layer_3': 51, 'learning_rate': 0.00047861799576722614}


C:\Users\jean_\AppData\Local\Temp\ipykernel_2328\2225789887.py:55: ExperimentalWarning:

plot_timeline is experimental (supported from v3.2.0). The interface can change in the future.



In [ ]:
# Limpeza do ambiente do TensorFlow
clear_session()

if TRAIN != True:
    # Carrega os melhores hiperparâmetros do modelo a partir do arquivo JSON
    with open(f"{model_path}encoder_decoder_hyperparameters.json", "r") as f:
        best_params = json.load(f)

    # Imprimir os melhores hiperparâmetros
    print("Hiperparâmetros Carregados:")
    for key, value in best_params.items(): print(f"{key}: {value}")

    # Use os melhores parâmetros para treinar o modelo final
    best_num_units_layer_0 = best_params["num_units_layer_0"]
    best_num_units_layer_1 = best_params["num_units_layer_1"]
    best_num_units_layer_2 = best_params["num_units_layer_2"]
    best_num_units_layer_3 = best_params["num_units_layer_3"]
    best_learning_rate = best_params["learning_rate"]
    
    # Definição do modelo
    encoder_decoder = Sequential([
        LSTM(best_num_units_layer_0, activation='relu', input_shape=input_shape),
        RepeatVector(past_history),
        LSTM(best_num_units_layer_1, activation='relu', return_sequences=True),
        TimeDistributed(Dense(best_num_units_layer_2, activation='relu')),
        Flatten(),
        Dense(best_num_units_layer_3, activation='relu'),
        Dense(1)
    ])

    # Definição de Callbacks
    model_checkpoint = ModelCheckpoint(f'{model_path}encoder_decoder.h5', save_best_only=True)
    optimizer = Adam(learning_rate=best_learning_rate, amsgrad=True)

    # Compilação do modelo
    encoder_decoder.compile(loss=loss, optimizer=optimizer, metrics=metric)

    # Treinamento do modelo
    history = encoder_decoder.fit(train, epochs=120, validation_data=validation, callbacks=[early_stopping, model_checkpoint])

    # Plotagem da história de treinamento
    plot_training_history(history)

Hiperparâmetros Carregados:
num_units_layer_0: 162
num_units_layer_1: 103
num_units_layer_2: 113
num_units_layer_3: 51
learning_rate: 0.00047861799576722614
Epoch 1/120
250/250 [==============================] - 132s 510ms/step - loss: 0.0032 - root_mean_squared_error: 0.0532 - val_loss: 0.0016 - val_root_mean_squared_error: 0.0399
Epoch 2/120
250/250 [==============================] - 87s 347ms/step - loss: 0.0012 - root_mean_squared_error: 0.0346 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0384
Epoch 3/120
250/250 [==============================] - 85s 340ms/step - loss: 0.0012 - root_mean_squared_error: 0.0347 - val_loss: 0.0016 - val_root_mean_squared_error: 0.0400
Epoch 4/120
250/250 [==============================] - 88s 353ms/step - loss: 0.0010 - root_mean_squared_error: 0.0322 - val_loss: 0.0013 - val_root_mean_squared_error: 0.0365
Epoch 5/120
250/250 [==============================] - 86s 346ms/step - loss: 0.0011 - root_mean_squared_error: 0.0328 - val_loss: 0.0027 

In [ ]:
# Carregamento do modelo com melhor desempenho
encoder_decoder = load_model(f'{model_path}encoder_decoder.h5')

# Geração de previsões usando o modelo treinado
forecast = encoder_decoder.predict(X_test)
encoder_decoder_pred = scaler_y.inverse_transform(forecast)

20/20 [==============================] - 2s 93ms/step


In [ ]:
# Exibição das métricas de avaliação e do gráfico de comparação
resultado = metric_display(y_test_inv, encoder_decoder_pred, df_final.index[valid_end_idx+past_history], df_final.index[test_end_idx-1])

# Adicionando as métricas de avaliação ao DataFrame, se o modelo já estiver no DataFrame, atualize as métricas, caso contrário, adicione uma nova linha
model_name = 'Encoder-Decoder'
if model_name in model_metrics['MODEL'].values:
    model_metrics.loc[model_metrics['MODEL'] == model_name] = [model_name] + resultado
else:
    model_metrics.loc[len(model_metrics)] = [model_name] + resultado

[Menor é melhor] RMSE: 263.4904
[Menor é melhor] MAE: 184.9837
[Menor é melhor] MAPE: inf%
[Menor é melhor] SMAPE: 14.0262% 
[Maior é melhor] R²: 0.7189 
[Maior é melhor] EVS: 0.7249 


# PLOT

In [ ]:
# Separar os dados de treinamento, validação e teste
train_plot = df_final[target][:train_end_idx]
cv_plot = df_final[target][train_end_idx:valid_end_idx]
test_plot = df_final[target][valid_end_idx:test_end_idx]
cat_plot = pd.Series(cat_pred.ravel(), index=test_plot.index[past_history:])
xgb_plot = pd.Series(xgb_pred.ravel(), index=test_plot.index[past_history:])
lgbm_plot = pd.Series(lstm_pred.ravel(), index=test_plot.index[past_history:])
stacked_lgbm_plot = pd.Series(multivariate_stacked_lstm_pred.ravel(), index=test_plot.index[past_history:])
cnn_plot = pd.Series(multivariate_cnn_pred.ravel(), index=test_plot.index[past_history:])
cnn_lstm_plot = pd.Series(multivariate_cnn_lstm_pred.ravel(), index=test_plot.index[past_history:])
mlp_plot = pd.Series(multivariate_mlp_pred.ravel(), index=test_plot.index[past_history:])
encoder_decoder_plot = pd.Series(encoder_decoder_pred.ravel(), index=test_plot.index[past_history:])

# Criar traços para os dados de treinamento e teste
train_trace = go.Scatter(x=train_plot.index, y=train_plot, mode='lines', name='Dados de Treinamento', line=dict(color='blue'))
cv_trace = go.Scatter(x=cv_plot.index, y=cv_plot, mode='lines', name='Dados de Validação', line=dict(color='orange'))
test_trace = go.Scatter(x=test_plot.index, y=test_plot, mode='lines', name='Dados de Teste', line=dict(color='green'))
cat_trace = go.Scatter(x=cat_plot.index, y=cat_plot, mode='lines', name='Previsões (CatBoost)', line=dict(color='darkblue'))
xgb_trace = go.Scatter(x=xgb_plot.index, y=xgb_plot, mode='lines', name='Previsões (XGBoost)', line=dict(color='red'))
lgbm_trace = go.Scatter(x=lgbm_plot.index, y=lgbm_plot, mode='lines', name='Previsões (LSTM)', line=dict(color='purple'))
multi_lgbm_trace = go.Scatter(x=stacked_lgbm_plot.index, y=stacked_lgbm_plot, mode='lines', name='Previsões (Multivariate Stacked LSTM)', line=dict(color='black'))
cnn_trace = go.Scatter(x=cnn_plot.index, y=cnn_plot, mode='lines', name='Previsões (CNN)', line=dict(color='pink'))
cnn_lstm_trace = go.Scatter(x=cnn_lstm_plot.index, y=cnn_lstm_plot, mode='lines', name='Previsões (CNN-LSTM)', line=dict(color='brown'))
mlp_trace = go.Scatter(x=mlp_plot.index, y=mlp_plot, mode='lines', name='Previsões (MLP)', line=dict(color='gray'))
encoder_decoder_trace = go.Scatter(x=encoder_decoder_plot.index, y=encoder_decoder_plot, mode='lines', name='Previsões (Encoder-Decoder)', line=dict(color='cyan'))


# Definir o layout do gráfico
layout = go.Layout(title='Previsão de Demanda',
                   xaxis=dict(title='Data e Hora'),
                   yaxis=dict(title='Demanda [kVA]]'))

# Criar a figura com os traços de treinamento, teste e previsões do melhor modelo
fig = go.Figure(data=[train_trace, cv_trace, test_trace, cat_trace, xgb_trace, lgbm_trace, multi_lgbm_trace, cnn_trace, cnn_lstm_trace, mlp_trace, encoder_decoder_trace], layout=layout)

# Plotar o gráfico
pyo.iplot(fig)

In [ ]:
# Apresentar as métricas de avaliação de cada modelo, ordenadas pelo RMSE
model_metrics.sort_values(by=['RMSE'])

,MODEL,RMSE,MAE,MAPE,SMAPE,R²,EVS
0,CatBoost,248.114608,168.757841,inf,13.283654,0.750752,0.754799
3,LSTM,248.332546,167.792627,inf,12.689486,0.750314,0.751684
4,Stacked LSTM,257.736990,178.212447,inf,14.104861,0.731044,0.763150
5,CNN,257.736990,178.212447,inf,14.104861,0.731044,0.763150
2,XGBoost,259.659777,180.328719,inf,13.757312,0.727016,0.734428
1,LightGBM,261.702929,183.620701,inf,13.817293,0.722703,0.737120
8,Encoder-Decoder,263.490365,184.983712,inf,14.026231,0.718902,0.724918
6,CNN-LSTM,266.212714,197.214106,inf,14.825444,0.713064,0.715972
7,MLP,266.689699,194.595776,inf,14.388776,0.712035,0.724477
